In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/plant-pathology-2020-fgvc7/sample_submission.csv
/kaggle/input/plant-pathology-2020-fgvc7/train.csv
/kaggle/input/plant-pathology-2020-fgvc7/test.csv
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_1743.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_262.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1524.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1336.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1461.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1797.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_1499.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1170.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_1259.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_422.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_636.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_1395.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_1800.jpg
/kaggle/input/plant-patholog

In [ ]:
import pandas as pd
path = '/kaggle/input/plant-pathology-2020-fgvc7/'
img_path = path + 'images'
# LOAD THE DATASET
train_df = pd.read_csv(path + 'train.csv')
test_df = pd.read_csv(path + 'test.csv')
sample = pd.read_csv(path + 'sample_submission.csv')
# GET THE IMAGE FILE NAME
train_df['img_path'] = train_df['image_id'] + '.jpg'
test_df['img_path'] = test_df['image_id'] + '.jpg'
train_df.head()

,image_id,healthy,multiple_diseases,rust,scab,img_path
0,Train_0,0,0,0,1,Train_0.jpg
1,Train_1,0,1,0,0,Train_1.jpg
2,Train_2,1,0,0,0,Train_2.jpg
3,Train_3,0,0,1,0,Train_3.jpg
4,Train_4,1,0,0,0,Train_4.jpg


In [ ]:
train_label = train_df.melt(id_vars=['image_id', 'img_path'])
train_label = train_label[train_label['value'] == 1]
train_label['id'] = [int(i[1]) for i in train_label['image_id'].str.split('_')]
train_label = train_label.sort_values('id').reset_index()
train_df['label'] = train_label['variable']
train_df = train_df[train_df.columns[[0, 5, 1, 2, 3, 4, 6]]]
print(train_label.shape)
train_df.head()

(1821, 6)


,image_id,img_path,healthy,multiple_diseases,rust,scab,label
0,Train_0,Train_0.jpg,0,0,0,1,scab
1,Train_1,Train_1.jpg,0,1,0,0,multiple_diseases
2,Train_2,Train_2.jpg,1,0,0,0,healthy
3,Train_3,Train_3.jpg,0,0,1,0,rust
4,Train_4,Train_4.jpg,1,0,0,0,healthy


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label_encoded = le.fit_transform(train_df['label'])
train_df['label_encoded'] = label_encoded
train_df.head()

,image_id,img_path,healthy,multiple_diseases,rust,scab,label,label_encoded
0,Train_0,Train_0.jpg,0,0,0,1,scab,3
1,Train_1,Train_1.jpg,0,1,0,0,multiple_diseases,1
2,Train_2,Train_2.jpg,1,0,0,0,healthy,0
3,Train_3,Train_3.jpg,0,0,1,0,rust,2
4,Train_4,Train_4.jpg,1,0,0,0,healthy,0


In [ ]:
from torchvision import transforms
import torch
import matplotlib.pyplot as plt
class Plants():
    def __init__(self, data_frame, root_dir, transform=None):
        self.data_frame = data_frame
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.data_frame)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 1])
        image = Image.open(img_name)
        label = self.data_frame.iloc[idx, -1]
        
        if self.transform:
            image = self.transform(image)
    
        return (image, label)
plant_train = Plants(
    data_frame=train_df,
    root_dir=path + 'images',
    transform=transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)
examples = enumerate(plant_train)
examples